# Create Cube environment


In [1]:
# Mounting the google drive to google colab in order to load the data files directly from it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp -a '/content/drive/MyDrive/Action learning/DQN/.' .

# Set up

In [3]:
import time

def simplify_move_str(move_str):
    move_list = move_str.strip().split(' ')
    replacement = 1
    # simplified = []
    while replacement != 0:
        replacement = 0
        i = 1
        simplified = []
        while i < len(move_list) and len(move_list) > 1:
            left, right = move_list[i-1], move_list[i]
            if left[0] == right[0]:
                moves_len = len(left) + len(right)
                print(moves_len, left, right)
                if moves_len in [2, 4] and '2' not in left and '2' not in right:
                    simplified.append(left[0] + "2")
                elif moves_len == 3 and ("'" in left or "'" in right):
                    replacement += 1
                else:
                    simplified.append(left)
                    simplified.append(right)
                i+=2
            else:
                simplified.append(left)
                i += 1
        print(simplified)
        if simplified == []: return simplified
        temp = move_list.copy()
        move_list = simplified.copy()
        time.sleep(1)
    simplified.append(temp[-1])
    return simplified


In [4]:
# simplify_move_str("R R' B2 B' R2 R' R B B' L L L' L L B L'")

In [52]:
import numpy as np

def flatten(t):
    return [item for sublist in t for item in sublist]

def textfile_to_list(filepath):
    with open(filepath) as f:
        lines = f.read().splitlines() 
    return lines

def get_algs(filename):
    with open(filename) as f:
        lines = f.readlines()

    moves = []
    for l in lines[1:]:
        moves.append(l.strip().split(';')[-1])

    return moves

def extract_scramble_list(filepath):
    with open(filepath) as f:
        lines = f.readlines()

    scramble_list = []
    for l in lines:
        scramble = l.strip().split('.')[1][1:]
        scramble_list.append(scramble)
    
    return scramble_list

PLL_scrambles = '/content/drive/MyDrive/Action learning/PLL_scrambles_unique.txt'
OLL_scrambles = '/content/drive/MyDrive/Action learning/OLL_scrambles_unique.txt'
OLL_algs = '/content/drive/MyDrive/Action learning/OLL algs.csv'
PLL_algs =  '/content/drive/MyDrive/Action learning/PLL algs.csv'
F2L1_scrambles = '/content/drive/MyDrive/Action learning/F2L_scrambles_unique.txt'
F2L2_scrambles = '/content/drive/MyDrive/Action learning/F2L2_scrambles_unique.txt'
F2L3_scrambles = '/content/drive/MyDrive/Action learning/F2L3_scrambles_unique.txt'
F2L4_scrambles = '/content/drive/MyDrive/Action learning/F2L4_scrambles_unique.txt'
Daisy_scrambles = '/content/drive/MyDrive/Action learning/daisy_scrambles_unique_new.txt'
Full_scrambles = '/content/drive/MyDrive/Action learning/15000_full_scrambles.txt'

oll_moves = get_algs(OLL_algs)
pll_moves = get_algs(PLL_algs)

pll_scrambles = textfile_to_list(PLL_scrambles)
oll_scrambles = textfile_to_list(OLL_scrambles)
f2l1_scrambles = textfile_to_list(F2L1_scrambles)
f2l2_scrambles = textfile_to_list(F2L2_scrambles)
f2l3_scrambles = textfile_to_list(F2L3_scrambles)
f2l4_scrambles = textfile_to_list(F2L4_scrambles)
daisy_scrambles = textfile_to_list(Daisy_scrambles)
full_scramble_list = extract_scramble_list(Full_scrambles)

cube_colors = {0:"W",1:"G",2:"R",3:'B',4:"O",5:'Y'}
move_list = ["R", "L", "U", "D", "F", "B", "Rp", "Lp", "Up", "Dp", "Fp", "Bp"]
daisy_moves= ["R", "L", "F", "B", "Rp", "Lp", "Fp", "Bp"]
daisy_moves= ["R", "L", "F", "B", "U", "D"]
cross_from_daisy_moves = ["R2", "L2", "U", "F2", "B2", "Up"]

# F2L1 - Front-Right F2L
F2L1_moves = ["U", "U'", "R U R'", "R U' R'", "R' U R", "R' U' R", "L U L'", 
              "L U' L'", "L' U L", "L' U' L", "F' U F", "F' U' F", "B U B'",
              "B U' B'", "B' U B", "B' U' B", "F U F'", "F U' F'"]

# F2L2 - Back-Right F2L
F2L2_moves = ["U", "U'", "R' U R", "R' U' R", "L U L'", 
              "L U' L'", "L' U L", "L' U' L", "B U B'",
              "B U' B'", "B' U B", "B' U' B", "F U F'", "F U' F'"]

# F2L3 - Back-Left F2L
F2L3_moves = ["U", "U'", "L U L'", "L U' L'", "L' U L", "L' U' L", 
              "B' U B", "B' U' B", "F U F'", "F U' F'"]

# F2L4 - Front-left F2L
F2L4_moves = ["U", "U'", "L' U L", "L' U' L", "F U F'", "F U' F'"]


SCRAMBLE_SIZE = 30
MAX_EXPLO = 70
CROSS_SUCCESS = 50

def print_face(face):
    for i in range(3):
        print(cube_colors[face[i*3]],cube_colors[face[i*3+1]],cube_colors[face[i*3+2]])

In [6]:
daisy_scrambles[:3]

["D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F U R F' D' Bp Bp R Fp R Fp",
 "R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F' L2 D' B U B L F Lp Rp Lp",
 "F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' R2 F2 L D2 U L R Bp Bp R R"]

# Classes definition

## Parent cube class

In [7]:
class SpeedCube():
    def __init__(self):
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3, np.ones(9)*4,
                              np.ones(9)*5], dtype=np.int32)
        self.solved_cross = self.get_yellow_edges()
        self.move_list = move_list
        self._episode_ended = False
        self._state = 0
        self._reward = 0
        self._solved_episodes = 0
        self._failed_episodes = 0
        self.scramble_size = 1

    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        # scramble_s = np.random.randint(1, SCRAMBLE_SIZE)
        scramble_s = np.random.randint(max(1, self.scramble_size-3), self.scramble_size+1)
        random_actions = np.random.randint(len(self.move_list), size=scramble_s)
        self._initial_scramble = [self.move_list[i] for i in random_actions]
        self.scramble(self._initial_scramble)
        return self.get_yellow_edges()
    
    def step(self, action):
        action = self.move_list[action]
        if action != "": getattr(self, action)()
        yellow_edges = self.get_yellow_edges()

        if np.all(self.solved_cross == yellow_edges):
            step_reward = 20
            self._episode_ended = True
            print('cross solved:', self._state+1)
        else:
            step_reward = -1

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                print(f'Number of cross solved ath this level {self._solved_episodes}/{CROSS_SUCCESS}')
                self._solved_episodes +=1
            else:
                # print('no solution found:', self._state)
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
            if self._solved_episodes == CROSS_SUCCESS:
                self.scramble_size = min(self.scramble_size+1, SCRAMBLE_SIZE)
                print("scramble size increase to", self.scramble_size)
                self._solved_episodes = 0
        return yellow_edges, float(step_reward), self._episode_ended, 'no'
        # else:
        #     return yellow_edges, float(step_reward), self._episode_ended, ''
        

        # 0: blanc
        # 1: vert
        # 2: red
        # 3: blue
        # 4: orange
        # 5: yellow
    def R(self):
        # permute corners
        self.status[2][0],self.status[2][2],self.status[2][8],self.status[2][6]=self.status[2][6],self.status[2][0],self.status[2][2],self.status[2][8]
        # permute edges
        self.status[2][1],self.status[2][5],self.status[2][7],self.status[2][3]=self.status[2][3],self.status[2][1],self.status[2][5],self.status[2][7]
        for i in range (3):
            self.status[0][i*3+2],self.status[1][i*3+2],self.status[5][i*3+2],self.status[3][6-3*i]=self.status[1][i*3+2],self.status[5][i*3+2],self.status[3][6-3*i],self.status[0][i*3+2]
            
    def L(self):
      # permute corners
        self.status[4][0],self.status[4][2],self.status[4][8],self.status[4][6]=self.status[4][6],self.status[4][0],self.status[4][2],self.status[4][8]
        # permute edges
        self.status[4][1],self.status[4][5],self.status[4][7],self.status[4][3]=self.status[4][3],self.status[4][1],self.status[4][5],self.status[4][7]
        for i in range (3):
            self.status[0][i*3],self.status[1][i*3],self.status[5][i*3],self.status[3][8-i*3]=self.status[3][8-i*3],self.status[0][i*3],self.status[1][i*3],self.status[5][i*3]
    def U(self):
        # permute corners
        self.status[0][0],self.status[0][2],self.status[0][8],self.status[0][6]=self.status[0][6],self.status[0][0],self.status[0][2],self.status[0][8]
        # permute edges
        self.status[0][1],self.status[0][5],self.status[0][7],self.status[0][3]=self.status[0][3],self.status[0][1],self.status[0][5],self.status[0][7]
        
        for i in range (3):
            self.status[1][i],self.status[4][i],self.status[3][i],self.status[2][i]=self.status[2][i],self.status[1][i],self.status[4][i],self.status[3][i]
    def D(self):
      # permute corners
        self.status[5][0],self.status[5][2],self.status[5][8],self.status[5][6]=self.status[5][6],self.status[5][0],self.status[5][2],self.status[5][8]
        # permute edges
        self.status[5][1],self.status[5][5],self.status[5][7],self.status[5][3]=self.status[5][3],self.status[5][1],self.status[5][5],self.status[5][7]
        for i in range (6,9):
            self.status[1][i],self.status[2][i],self.status[3][i],self.status[4][i]=self.status[4][i],self.status[1][i],self.status[2][i],self.status[3][i]
    def F(self):
      # permute corners
        self.status[1][0],self.status[1][2],self.status[1][8],self.status[1][6]=self.status[1][6],self.status[1][0],self.status[1][2],self.status[1][8]
        # permute edges
        self.status[1][1],self.status[1][5],self.status[1][7],self.status[1][3]=self.status[1][3],self.status[1][1],self.status[1][5],self.status[1][7]
        for i in range (3):
            self.status[0][6+i],self.status[2][i*3],self.status[5][2-i],self.status[4][8-i*3]=self.status[4][8-i*3],self.status[0][6+i],self.status[2][i*3],self.status[5][2-i]
    def B(self):
      # permute corners
        self.status[3][0],self.status[3][2],self.status[3][8],self.status[3][6]=self.status[3][6],self.status[3][0],self.status[3][2],self.status[3][8]
        # permute edges
        self.status[3][1],self.status[3][5],self.status[3][7],self.status[3][3]=self.status[3][3],self.status[3][1],self.status[3][5],self.status[3][7]
        for i in range (3):
            self.status[0][2-i],self.status[4][3*i],self.status[5][6+i],self.status[2][8-3*i]=self.status[2][8-3*i],self.status[0][2-i],self.status[4][3*i],self.status[5][6+i]
    def R2(self):
        self.R()
        self.R()
    def L2(self):
        self.L()
        self.L()
    def F2(self):
        self.F()
        self.F()
    def B2(self):
        self.B()
        self.B()
    def U2(self):
        self.U()
        self.U()
    def D2(self):
        self.D()
        self.D()

    def Rp(self):
        self.R()
        self.R()
        self.R()
    def Lp(self):
        self.L()
        self.L()
        self.L()
    def Fp(self):
        self.F()
        self.F()
        self.F()
    def Bp(self):
        self.B()
        self.B()
        self.B()
    def Up(self):
        self.U()
        self.U()
        self.U()
    def Dp(self):
        self.D()
        self.D()
        self.D()

    def z2(self):
        # probably correct
        temp = self.status[0].copy()
        self.status[0] = self.status[5][::-1] 
        self.status[5] = temp[::-1]
        # probably incorrect - to be checked
        idx = [6, 7, 8, 3, 4, 5, 0, 1, 2] 
        temp = self.status[2].copy()
        self.status[2] =  self.status[4][idx] 
        self.status[4] = temp[idx] 
        temp = self.status[1].copy()
        self.status[1] = temp[::-1]
        temp =  self.status[3].copy()
        self.status[3] = temp[::-1]

    # takes a scramble as a list of moves or a string
    def scramble(self,scramble):
        if isinstance(scramble, list):
            moves = [s.replace("'", "p") for s in scramble]
        else:
            moves=scramble.replace("'","p").split(' ')
        for i in moves:
            if i != "": getattr(self, i)()


    def show(self):        
        face=self.status[3]
        for i in range(3):
            print("      ",cube_colors[face[8-i*3]],cube_colors[face[7-i*3]],cube_colors[face[6-i*3]], "      " )
        print("")
        f1,f2,f3=self.status[4],self.status[0],self.status[2]
        for j in range(3):
            print(cube_colors[f1[6+j]],cube_colors[f1[3+j]],cube_colors[f1[j]],'',
                  cube_colors[f2[j*3]],cube_colors[f2[j*3+1]],cube_colors[f2[j*3+2]],'',
                  cube_colors[f3[2-j]],cube_colors[f3[5-j]],cube_colors[f3[8-j]])
        face=self.status[1]
        print("")
        for i in range(3):
            print("      ",cube_colors[face[i*3]],cube_colors[face[i*3+1]],cube_colors[face[i*3+2]], "      " )
        print("")
        face=self.status[5]
        for i in range(3):
            print("      ",cube_colors[face[i*3]],cube_colors[face[i*3+1]],cube_colors[face[i*3+2]], "      " )


    # Get list of edges in a list, each edge is composed of 2 faces:
    # DF, DR, DB, DL, FR, BR, BL, FL, UF, UR, UB, UL
    def get_edges(self):
        self.edges = [# Top (white) layer edges
                      [self.status[0][7], self.status[1][1]], #UF
                      [self.status[0][5], self.status[2][1]], #UR
                      [self.status[0][1], self.status[3][1]], #UB
                      [self.status[0][3], self.status[4][1]], #UL
                      # E layer edges
                      [self.status[1][3], self.status[4][5]], #FR
                      [self.status[3][5], self.status[4][3]], #BR
                      [self.status[3][3], self.status[2][5]], #BL                   
                      [self.status[1][5], self.status[2][3]], #FL
                      # Bottom layer (yellow) edges                     
                      [self.status[5][1], self.status[1][7]], #DF
                      [self.status[5][3], self.status[4][7]], #DR
                      [self.status[5][7], self.status[3][7]], #DB
                      [self.status[5][5], self.status[2][7]]  #DL
                      ]
        return self.edges
    
    def get_white_edges(self):
        self.white_edges = [e if 0 in e else '' for e in self.get_edges()]
        return self.white_edges

    def get_yellow_edges(self):
        self.yellow_edges_new = [e if 5 in e else [-1,-1] for e in self.get_edges()]
        return np.array(self.yellow_edges_new).flatten()

  

        # 0: blanc
        # 1: vert
        # 2: red
        # 3: blue
        # 4: orange
        # 5: yellow

    # Get list of corners in a list, each corner is composed of 3 faces:
    # DFR, DBR, DBL, DFL, UFR, UBR, UBL, UFL
    def get_corners(self):
        self.corners = [[self.status[1][8], self.status[2][6], self.status[5][2]], #DFR
                        [self.status[2][8], self.status[3][6], self.status[5][8]],
                        [self.status[3][8], self.status[4][6], self.status[5][6]],
                        [self.status[4][8], self.status[1][6], self.status[5][0]],
                        [self.status[1][2], self.status[2][0], self.status[0][8]],
                        [self.status[2][2], self.status[3][0], self.status[0][2]],
                        [self.status[3][2], self.status[4][0], self.status[0][0]],
                        [self.status[4][2], self.status[1][0], self.status[0][6]]
                        ]
        return self.corners 



    def get_last_layer(self):
        edges = self.get_edges()
        corners = self.get_corners()
        ll_state = edges[:4]
        ll_state.extend(corners[4:]) 
        return np.array(flatten(ll_state))
    
    def get_oll_state(self):
        return np.array([x if x==0 else -1 for x in self.get_last_layer()])
    
    def get_pll_state(self):
        pll_state = np.concatenate([self.status[i+1][:3] for i in range(4)])
        return np.array(pll_state).flatten()
    
    def get_f2l_state(self):
        edges = self.get_edges()
        corners = self.get_corners()
        f2l_state = edges[: 8]
        f2l_state.extend(corners)
        return f2l_state
    
    def get_f2l1(self):
        f2l1_state = []
        f2l1_edge = [1, 2] 
        f2l1_corner = [1, 2, 5]
        for p in self.get_f2l_state():
            if set(p) == set(f2l1_edge) or set(p) == set(f2l1_corner):
                f2l1_state.append(p)
            else:
                f2l1_state.append(list(-np.ones(len(p))))
        return np.array(flatten(f2l1_state))
    
    def get_f2l2(self):
        f2l2_state = []
        f2l2_edge = [2, 3] 
        f2l2_corner = [2, 3, 5]
        for p in self.get_f2l_state():
            if set(p) == set(f2l2_edge) or set(p) == set(f2l2_corner):
                f2l2_state.append(p)
            else:
                f2l2_state.append(list(-np.ones(len(p))))
        return np.array(flatten(f2l2_state))
    
    def get_f2l3(self):
        f2l3_state = []
        f2l3_edge = [3, 4] 
        f2l3_corner = [3, 4, 5]
        for p in self.get_f2l_state():
            if set(p) == set(f2l3_edge) or set(p) == set(f2l3_corner):
                f2l3_state.append(p)
            else:
                f2l3_state.append(list(-np.ones(len(p))))
        return np.array(flatten(f2l3_state))
    
    def get_f2l4(self):
        f2l4_state = []
        f2l4_edge = [4, 1] 
        f2l4_corner = [4, 1, 5]
        for p in self.get_f2l_state():
            if set(p) == set(f2l4_edge) or set(p) == set(f2l4_corner):
                f2l4_state.append(p)
            else:
                f2l4_state.append(list(-np.ones(len(p))))
        return np.array(flatten(f2l4_state))
    
    def get_f2lpart1(self):
        f2l1_state = []
        f2l1_edge = [1, 2] 
        f2l1_corner = [1, 2, 5]
        f2l_pieces = [[1, 2], [2, 3], [1, 2, 5], [2, 3, 5] ]
        for p in self.get_f2l_state():
            if np.any([set(p) == set(x) for x in f2l_pieces]):
                f2l1_state.append(p)
            else:
                f2l1_state.append(list(-np.ones(len(p))))
        return np.array(flatten(f2l1_state))
    
    def get_daisy(self):
        self.daisy_pieces = flatten([e if 5 in e else [-1,-1] for e in self.get_edges()])
        self.daisy_pieces = np.array([e if e == 5 else -1 for e in self.daisy_pieces])
        return self.daisy_pieces


## Cube subclasses

In [14]:

class PLL_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_pll = self.get_pll_state()
        self.move_list = pll_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        auf = np.random.randint(0, 3)
        scramble_pll = np.random.choice(pll_scrambles)     
        for _ in range(auf):
            scramble_pll += " U"
        self.scramble(scramble_pll)
        return self.get_pll_state()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        pll_state = self.get_pll_state()
        if np.all(self.solved_pll == pll_state):
            step_reward = 20
            self._episode_ended = True
            print('PLL solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -5

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                # print('no solution found:', self._state)
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return pll_state, float(step_reward), self._episode_ended, ''
        # else:
        #     return yellow_edges, float(step_reward), self._episode_ended, ''
        
class OLL_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_oll = self.get_oll_state()
        self.move_list = oll_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        auf = np.random.randint(0, 3)
        scramble_oll = np.random.choice(oll_scrambles)     
        for _ in range(auf):
            scramble_oll += " U"
        self.scramble(scramble_oll)
        return self.get_oll_state()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        oll_state = self.get_oll_state()
        if np.all(self.solved_oll == oll_state):
            step_reward = 20
            self._episode_ended = True
            print('OLL solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -5

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                # print('no solution found:', self._state)
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return oll_state, float(step_reward), self._episode_ended, ''
        # else:
        #     return yellow_edges, float(step_reward), self._episode_ended, ''

class F2L1_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_f2l1 = self.get_f2l1()
        self.move_list = F2L1_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_f2l1 = np.random.choice(f2l1_scrambles)     
        self.scramble(scramble_f2l1)
        return self.get_f2l1()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        f2l1_state = self.get_f2l1()
        if np.all(self.solved_f2l1 == f2l1_state):
            step_reward = 20
            self._episode_ended = True
            print('F2L1 solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -2

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return f2l1_state, float(step_reward), self._episode_ended, ''
            



class F2L2_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_f2l2 = self.get_f2l2()
        self.move_list = F2L2_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_f2l2 = np.random.choice(f2l2_scrambles)     
        self.scramble(scramble_f2l2)
        return self.get_f2l2()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        f2l2_state = self.get_f2l2()
        if np.all(self.solved_f2l2 == f2l2_state):
            step_reward = 20
            self._episode_ended = True
            print('F2L2 solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -2

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return f2l2_state, float(step_reward), self._episode_ended, ''

class F2L3_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_f2l3 = self.get_f2l3()
        self.move_list = F2L3_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_f2l3 = np.random.choice(f2l3_scrambles)     
        self.scramble(scramble_f2l3)
        return self.get_f2l3()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        f2l3_state = self.get_f2l3()
        if np.all(self.solved_f2l3 == f2l3_state):
            step_reward = 20
            self._episode_ended = True
            print('F2L3 solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -2

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return f2l3_state, float(step_reward), self._episode_ended, ''

class F2L4_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_f2l4 = self.get_f2l4()
        self.move_list = F2L4_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_f2l4 = np.random.choice(f2l4_scrambles)     
        self.scramble(scramble_f2l4)
        return self.get_f2l4()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        f2l4_state = self.get_f2l4()
        if np.all(self.solved_f2l4 == f2l4_state):
            step_reward = 20
            self._episode_ended = True
            print('F2L4 solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -2

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return f2l4_state, float(step_reward), self._episode_ended, ''

class F2Lpart1_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_f2l1 = self.get_f2lpart1()
        self.move_list = F2L1_moves
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_f2l1 = np.random.choice(f2l1_scrambles)     
        self.scramble(scramble_f2l1)
        return self.get_f2lpart1()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        f2l1_state = self.get_f2lpart1()
        if np.all(self.solved_f2l1 == f2l1_state):
            step_reward = 20
            self._episode_ended = True
            print('F2L1 solved:', self._state+1)
        elif action in ["U", "U'"]:
            step_reward = -1
        else:
            step_reward = -2

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return f2l1_state, float(step_reward), self._episode_ended, ''


class Daisy_cube(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_daisy = self.get_solved_daisy()
        self.move_list = daisy_moves
        # self.move_list = move_list
    
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        scramble_s = np.random.randint(1, SCRAMBLE_SIZE)
        # scramble_s = np.random.randint(max(1, self.scramble_size-3), self.scramble_size+1)
        random_actions = np.random.randint(len(self.move_list), size=scramble_s)
        self._initial_scramble = [move_list[i] for i in random_actions]
        # self._initial_scramble = np.random.choice(full_scramble_list)
        self.scramble(self._initial_scramble)
        return self.get_daisy()

    def step(self, action):
        action = self.move_list[action]
        if action != "": self.scramble(action)
        daisy_state = self.get_daisy()
        if np.all(self.solved_daisy == daisy_state):
            step_reward = 20
            self._episode_ended = True
            print('Daisy solved:', self._state+1)
        else:
            step_reward = -1

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                # print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True
        return daisy_state, float(step_reward), self._episode_ended, ''

    def get_solved_daisy(self):
        daisy = - np.ones(24)
        for e in range(0, 7, 2):
            daisy[e] = 5
        return np.array(daisy)


class Cross_from_daisy(SpeedCube):
    def __init__(self):
        super().__init__()
        self.solved_cross = self.get_yellow_edges()
        self.move_list = cross_from_daisy_moves
    
     
    def reset(self):
        self._state = 0
        self._reward = 0
        self._episode_ended = False
        self.status = np.array([np.zeros(9)*3, np.ones(9), np.ones(9)*2, np.ones(9)*3,
                                np.ones(9)*4, np.ones(9)*5], dtype=np.int32)
        self.scramble(np.random.choice(daisy_scrambles))
        return self.get_yellow_edges()
    
    def step(self, action):
        action = self.move_list[action]
        if action != "": getattr(self, action)()
        yellow_edges = self.get_yellow_edges()

        if np.all(self.solved_cross == yellow_edges):
            step_reward = 20
            self._episode_ended = True
            print('cross solved:', self._state+1)
        else:
            step_reward = -1

        
        self._reward += step_reward
        self._state += 1

        if self._episode_ended or self._state > MAX_EXPLO:
            if self._episode_ended : 
                print('episode ended:', self._state)
                self._solved_episodes +=1
            else:
                # print('no solution found:', self._state)
                self._failed_episodes +=1
            # print(f'{self._solved_episodes}/{self._failed_episodes+self._solved_episodes} solved cross')
            self._episode_ended = True

        return yellow_edges, float(step_reward), self._episode_ended, 'no'

In [9]:
cube = Daisy_cube()
type(cube.reset())
print(cube.get_daisy())

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1  5  5 -1 -1  5 -1 -1]


In [ ]:
cube = F2L4_cube()
len(cube.move_list)

6

## Checking cube moves functions etc.

In [ ]:
cube = SpeedCube()
cube.scramble("R U")

In [ ]:
# move_list = ["R", "L", "U", "D", "F", "B", "Rp", "Lp", "Up", "Dp", "Fp", "Bp"]
cube.step(1)

cross solved: 2
episode ended: 3
1/1 solved cross


(array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
         1,  5,  4,  5,  3,  5,  2]), 10.0)

In [ ]:
np.array(cube.get_yellow_edges()).flatten()

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,  2, -1, -1,  5,
        1,  5,  4,  5,  3, -1, -1])

In [ ]:
np.all(np.array(cube.solved_cross).flatten() == np.array(cube.get_yellow_edges()).flatten())

False

In [ ]:
cube.step(12)

TimeStep(
{'discount': array(0., dtype=float32),
 'observation': array([1, 3, 5, 0, 0, 4, 4, 5, 0, 0, 1, 3, 0, 1, 5, 5, 0, 1, 4, 3, 2, 2,
       2, 5, 0, 3, 3, 1, 5, 4, 4, 3, 1, 2, 3, 2, 5, 4, 1, 2, 4, 2, 3, 1,
       4, 3, 1, 2, 4, 5, 2, 5, 0, 0], dtype=int32),
 'reward': array(-1., dtype=float32),
 'step_type': array(2, dtype=int32)})

In [ ]:
moves=["L",'R',"D","U",'D','R','U',"L",'L','D','R','U','U']
# moves="L"
moves = "F2 L F L2 D R2 D2 B2 D' F2 R2 B2 L2 D' B' D2 F' D R F' D2"

In [ ]:
cube.scramble(moves)

In [ ]:
with open("solutions_croix.txt") as f:
    lines = f.readlines()

FileNotFoundError: ignored

In [ ]:
len(lines)

NameError: ignored

In [ ]:
def revert_scramble(scramble: str):
    s = scramble.strip().split(' ')
    return [invert_move(x) for x in s[::-1]]

def invert_move(move: str):
    if len(move) == 0:
        return ""
    elif len(move) == 1:
        return move +"p"
    elif move[1] == '2':
        return move
    elif move[1] == "'" or move[1] == 'p':
        return move[0]
    else:
        return 'Invalid move - cannot invert'

In [ ]:
%%time

scramble_list = []
solution_list = []
for l in lines:
    solution_list.append(l.strip().split(' '))
    scramble_list.append(revert_scramble(l))

CPU times: user 2 s, sys: 112 ms, total: 2.11 s
Wall time: 2.12 s


In [ ]:
group_list = list(zip(scramble_list, solution_list))

In [ ]:
import pandas as pd

cross_df = pd.DataFrame(group_list, columns=['Scramble', 'Solution'])
cross_df.head()

,Scramble,Solution
0,"[B2, F2, Up, Lp, Rp]","[R, L, U, F2, B2]"
1,"[B2, F2, Up, Lp, Rp, B2]","[B2, R, L, U, F2, B2]"
2,"[L2, R2, D, L, Rp]","[R, L', D', R2, L2]"
3,"[L2, R2, Dp, L, Rp]","[R, L', D, R2, L2]"
4,"[B2, F2, U, Lp, Rp, B2]","[B2, R, L, U', F2, B2]"


In [ ]:
def scramble_standalone(scramble):
    cube = SpeedCube()
    cube.scramble(scramble)
    return str(cube.get_edges())

def get_white_edges(scramble):
    cube = SpeedCube()    
    cube.scramble(scramble)
    # cube.z2()
    all_edges = cube.get_edges()
    white_edges = [e if 0 in e else '' for e in all_edges]
    return str(white_edges)

def get_yellow_edges(scramble):
    cube = SpeedCube()    
    cube.scramble(scramble)
    # cube.z2()
    yellow_edges = [e if 5 in e else '' for e in cube.get_edges()]
    return str(yellow_edges)

In [ ]:
scramble = "L' D2 F2 L2 F L2 R2 U2 B L2 U2 B' D R U' B L D' R' U"
get_white_edges(scramble)

"[[2, 0], '', '', '', '', '', [0, 1], '', [3, 0], '', [0, 4], '']"

In [ ]:
%%time

# cross_df['status'] = cross_df['Scramble'].apply(get_white_edges)
cross_df['status'] = cross_df['Scramble'].apply(get_yellow_edges)

CPU times: user 50.6 s, sys: 2.12 s, total: 52.7 s
Wall time: 50.1 s


In [ ]:
cube = SpeedCube()
scramble = "F' R2 D' L2 B' U R' U2 R2 F' R2 B2 L2 B L2 U2 B D' F'"
cube.scramble(scramble)
# cube.z2()

In [ ]:
cube.show2()

       B O O       
       Y B G       
       Y R R       

O B G  O Y G  Y O G
B O G  R W B  W R R
G Y B  Y G B  W B W

       R W O       
       O G R       
       W G R       

       R Y B       
       O Y W       
       Y W W       


In [ ]:
edges_state = str(cube.get_yellow_edges())
cross_solution = cross_df[cross_df['status'] == edges_state]['Solution'].iloc[0]

In [ ]:
cross_solution

['D', "F'", "L'", 'B2', "D'"]

In [ ]:
cube.step("D'")

(['', '', '', '', '', '', '', '', [5, 1], [5, 4], [5, 3], [5, 2]], 10)

In [ ]:

cube.scramble(cross_solution)

In [ ]:
cross_df[cross_df['str_solution'].str.startswith("['F2', 'L', 'B', \"D'\", \"L'\"]")] 

,Scramble,Solution,status,str_solution
128942,"[L, D, Bp, Lp, F2]","[F2, L, B, D', L']","['', [0, 4], '', [0, 2], '', '', [3, 0], '', [...","['F2', 'L', 'B', ""D'"", ""L'""]"


In [ ]:
cross_df['str_solution'] = cross_df['Solution'].astype('str')
cross_df.head()

,Scramble,Solution,status,str_solution
0,"[B2, F2, Up, Lp, Rp]","[R, L, U, F2, B2]","[[0, 4], '', [0, 2], '', '', '', '', '', [0, 1...","['R', 'L', 'U', 'F2', 'B2']"
1,"[B2, F2, Up, Lp, Rp, B2]","[B2, R, L, U, F2, B2]","[[0, 4], '', [0, 3], '', '', '', '', '', [0, 1...","['B2', 'R', 'L', 'U', 'F2', 'B2']"
2,"[L2, R2, D, L, Rp]","[R, L', D', R2, L2]","[[0, 1], '', [0, 3], '', '', '', '', '', [0, 4...","['R', ""L'"", ""D'"", 'R2', 'L2']"
3,"[L2, R2, Dp, L, Rp]","[R, L', D, R2, L2]","[[0, 1], '', [0, 3], '', '', '', '', '', [0, 2...","['R', ""L'"", 'D', 'R2', 'L2']"
4,"[B2, F2, U, Lp, Rp, B2]","[B2, R, L, U', F2, B2]","[[0, 2], '', [0, 3], '', '', '', '', '', [0, 1...","['B2', 'R', 'L', ""U'"", 'F2', 'B2']"


In [ ]:
a.iloc[0]

"[[2, 0], '', '', '', '', '', [0, 1], '', [4, 0], '', '', [3, 0]]"

#Deep Q Daisy


## Training

In [15]:
model_path = '/content/drive/MyDrive/Action learning/daisy_model_nohack.h5'

In [16]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = Daisy_cube()

    n_games = 2000
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1, alpha=0.0005, input_dims=len(env.solved_daisy),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.999, batch_size=128, fname=model_path)

    scores = []
    eps_history = []
    # agent.load_model()

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)
    print("##### Best score:", best_score)

episode:  0 score -71.00:   average score -71.00
episode:  1 score -71.00:   average score -71.00
episode:  2 score -71.00:   average score -71.00
episode:  3 score -71.00:   average score -71.00
episode:  4 score -71.00:   average score -71.00
episode:  5 score -71.00:   average score -71.00
episode:  6 score -71.00:   average score -71.00
episode:  7 score -71.00:   average score -71.00
episode:  8 score -71.00:   average score -71.00
episode:  9 score -71.00:   average score -71.00
episode:  10 score -71.00:   average score -71.00
episode:  11 score -71.00:   average score -71.00
episode:  12 score -71.00:   average score -71.00
episode:  13 score -71.00:   average score -71.00
episode:  14 score -71.00:   average score -71.00
episode:  15 score -71.00:   average score -71.00
episode:  16 score -71.00:   average score -71.00
episode:  17 score -71.00:   average score -71.00
episode:  18 score -71.00:   average score -71.00
episode:  19 score -71.00:   average score -71.00
episode:  

KeyboardInterrupt: ignored

In [17]:
cube = Daisy_cube()
# obs = cube.reset()
cube.scramble("U' L2 F2 U' F2 U L2 B2 R2 F2 U2 L F' U' L' B F2 U B L")
# obs, _, _, _ = cube.step(0)
obs = cube.get_daisy()

In [18]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(daisy_moves[action])
    solution += daisy_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### Daisy SOLUTION #####')
print(solution)

D
0 (array([-1, -1, -1, -1, -1, -1,  5, -1, -1,  5, -1, -1,  5, -1, -1, -1, -1,
        5, -1, -1, -1, -1, -1, -1]), -1.0, False, '')
D
1 (array([-1, -1, -1, -1, -1, -1,  5, -1, -1,  5, -1, -1,  5, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  5]), -1.0, False, '')
R
2 (array([-1, -1, -1, -1, -1, -1,  5, -1, -1,  5, -1, -1, -1, -1, -1,  5, -1,
       -1, -1, -1, -1, -1,  5, -1]), -1.0, False, '')
R
3 (array([-1, -1, -1,  5, -1, -1,  5, -1, -1,  5, -1, -1, -1, -1,  5, -1, -1,
       -1, -1, -1, -1, -1, -1, -1]), -1.0, False, '')
R
4 (array([-1, -1,  5, -1, -1, -1,  5, -1, -1,  5, -1, -1, -1,  5, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1]), -1.0, False, '')
F
5 (array([ 5, -1,  5, -1, -1, -1,  5, -1, -1, -1, -1, -1, -1,  5, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1]), -1.0, False, '')
Daisy solved: 7
B
6 (array([ 5, -1,  5, -1,  5, -1,  5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1]), 20.0, True, '')

 ###### Daisy SOLUTION #####
D D R R R F B 


## Predictions from loaded model

In [20]:
from dqn_agent import DQNAgent 
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/Action learning/daisy_model_nohack.h5'


input_oll = 20
input_pll = 12
input_f2l2 = 40
input_cross = 24
actions_cross = 12
actions_oll = 59
actions_daisy = len(daisy_moves)
actions_f2l2 = len(F2L2_moves)
# daisy_solver = DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_cross,
#                       n_actions=actions_daisy, mem_size=1_000_000, epsilon_end=0.01,
#                       batch_size=64, fname=model_path)

daisy_solver = load_model(model_path)

 
def predict_daisy(scramble: str):
    daisy_cube = Daisy_cube()
    daisy_cube.scramble(scramble)
    obs = daisy_cube.get_daisy()
    i = 0
    solution = ""
    done = False
    while not done:
        action = np.argmax(daisy_solver.predict(obs[np.newaxis, :]))
        solution += daisy_cube.move_list[action] + " "
        next = daisy_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No daisy solution found for this scramble"
    else:
        return solution

In [22]:
predict_daisy("B' D' F2 D L2 R2 D L2 D2 U' L2 U2 R' F U' B L2 D L2 B")

Daisy solved: 7


'L D B U R R R '

#Deep Q Cross_from_daisy


## Training

In [39]:
model_path = '/content/drive/MyDrive/Action learning/DQN/Cross_from_daisy_model_new.h5'

In [40]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = Cross_from_daisy()

    n_games = 500
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_cross),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


episode:  0 score -71.00:   average score -71.00
cross solved: 44
episode ended: 44
episode:  1 score -23.00:   average score -47.00
episode:  2 score -71.00:   average score -55.00
episode:  3 score -71.00:   average score -59.00
episode:  4 score -71.00:   average score -61.40
episode:  5 score -71.00:   average score -63.00
cross solved: 33
episode ended: 33
episode:  6 score -12.00:   average score -55.71
episode:  7 score -71.00:   average score -57.62
episode:  8 score -71.00:   average score -59.11
episode:  9 score -71.00:   average score -60.30
episode:  10 score -71.00:   average score -61.27
cross solved: 8
episode ended: 8
episode:  11 score 13.00:   average score -55.08
cross solved: 7
episode ended: 7
episode:  12 score 14.00:   average score -49.77
episode:  13 score -71.00:   average score -51.29
episode:  14 score -71.00:   average score -52.60
cross solved: 28
episode ended: 28
episode:  15 score -7.00:   average score -49.75
cross solved: 16
episode ended: 16
episode

In [ ]:
cube = F2L2_cube()
# obs = cube.reset()
cube.scramble("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R'")
# obs, _, _, _ = cube.step(0)
obs = cube.get_f2l2()

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(F2L2_moves[action])
    solution += F2L2_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### F2L2 SOLUTION #####')
print(solution)

U'
0 (array([-1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  3.,  5.,  2., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -1.0, False, '')
B U B'
1 (array([-1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  2.,  5.,  3., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
U'
2 (array([-1., -1., -1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  2.,  5.,  3., -1., -1., -1., -1., -1.,
       -1.]), -1.0, False, '')
F U' F'
3 (array([-1., -1., -1., -1., -1., -1., -1., -1.,  3.,  2., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  2.,  5.,  3., -1., -1.,
       -1.

## Predictions from loaded model

In [ ]:
from dqn_agent import DQNAgent 

input_oll = 20
input_pll = 12
input_f2l2 = 40
actions_cross = 12
actions_oll = 59
actions_f2l2 = len(F2L2_moves)
f2l2_solver = dqn_agent.DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_f2l2,
                      n_actions=actions_f2l2, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=model_path)

f2l2_solver.load_model()

 
def predict_f2l2(scramble: str):
    f2l2_cube = F2L2_cube()
    f2l2_cube.scramble(scramble)
    obs = f2l2_cube.get_f2l2()
    i = 0
    solution = ""
    done = False
    while not done:
        action = f2l2_solver.choose_action(obs)
        solution += f2l2_cube.move_list[action] + " "
        next = f2l2_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No F2L2 solution found for this scramble"
    else:
        return solution

In [ ]:
predict_f2l2("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R'")

F2L2 solved: 6
episode ended: 6


"U' B U B' U' F U' F' L' U L R' U R "

#Deep Q Cross


In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DDQN/cross_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
from utils import plot_learning_curve, make_env
from dueling_ddqn_tf2 import Agent

if __name__ == '__main__':
    env = SpeedCube()

    n_games = 10000
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1, alpha=0.0005, input_dims=24,
                     n_actions=12, mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.9995, batch_size=64, fname=model_path)
    # agent = Agent(lr=0.003, gamma=0.99, n_actions=len(env.move_list), epsilon=1, 
    #              batch_size=128, input_dims=[len(env.solved_cross_new)], 
    #              fname=model_path)




    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done) # for DQNAgent
            # agent.store_transition(observation, action, reward, observation_, done) # for DDQN Agent
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score and i > 50:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)

    # filename = 'lunarlander.png'
    # x = [i+1 for i in range(len(scores))]
    # plot_learning_curve(x, scores, eps_history, filename)


episode:  0 score -51.00:   average score -51.00
episode:  1 score -51.00:   average score -51.00
episode:  2 score -51.00:   average score -51.00
episode:  3 score -51.00:   average score -51.00
episode:  4 score -51.00:   average score -51.00
cross solved: 1
episode ended: 1
episode:  5 score 20.00:   average score -39.17
episode:  6 score -51.00:   average score -40.86
episode:  7 score -51.00:   average score -42.12
cross solved: 1
episode ended: 1
episode:  8 score 20.00:   average score -35.22
episode:  9 score -51.00:   average score -36.80
episode:  10 score -51.00:   average score -38.09
episode:  11 score -51.00:   average score -39.17
episode:  12 score -51.00:   average score -40.08
episode:  13 score -51.00:   average score -40.86
episode:  14 score -51.00:   average score -41.53
episode:  15 score -51.00:   average score -42.12
episode:  16 score -51.00:   average score -42.65
episode:  17 score -51.00:   average score -43.11
episode:  18 score -51.00:   average score -43

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7ff93bb00320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
cube = SpeedCube()
obs = cube.reset()
cube.get_yellow_edges_new()

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,  4, -1, -1, -1, -1,  5,
        1, -1, -1,  5,  3,  5,  2])

In [ ]:
cube.show2()

       Y B B       
       Y B B       
       Y B B       

O O O  B W W  R R R
O O O  B W W  R R R
O O O  B W W  R R R

       W G G       
       W G G       
       W G G       

       G Y Y       
       G Y Y       
       G Y Y       


In [ ]:
cube.scramble("R L U F B R2 D B U2 L D R'")
obs, _,_,_ = cube.step(0)
cube._episode_ended = False

In [ ]:
i=0
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(move_list[action])
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1

Dp
0 (array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  3,  5, -1,
       -1,  1,  5,  5,  4,  2,  5]), -1.0, False, 'no')
Lp
1 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1, -1, -1,  5,  4,  2,  5]), -1.0, False, 'no')
D
2 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1,  5,  4,  2,  5, -1, -1]), -1.0, False, 'no')
Dp
3 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1, -1, -1,  5,  4,  2,  5]), -1.0, False, 'no')
D
4 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1,  5,  4,  2,  5, -1, -1]), -1.0, False, 'no')
Dp
5 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1, -1, -1,  5,  4,  2,  5]), -1.0, False, 'no')
D
6 (array([-1, -1, -1, -1, -1, -1, -1, -1,  1,  5, -1, -1, -1, -1,  3,  5, -1,
       -1,  5,  4,  2,  5, -1, -1]), -1.0, False, 'no')
Dp
7 (array([-1, -1, -1, -1, -1, -1, -1, -1,

In [ ]:
agent_loaded = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=24,
                     n_actions=12, mem_size=1_000_000, epsilon_end=0.01,
                     batch_size=64, fname=model_path)

agent_loaded.load_model()

#Deep Q F2L1


In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/F2L1_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = F2L1_cube()

    n_games = 300
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_f2l1),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


episode:  0 score -97.00:   average score -97.00
episode:  1 score -98.00:   average score -97.50
episode:  2 score -95.00:   average score -96.67
episode:  3 score -98.00:   average score -97.00
episode:  4 score -96.00:   average score -96.80
episode:  5 score -95.00:   average score -96.50
episode:  6 score -91.00:   average score -95.71
episode:  7 score -99.00:   average score -96.12
episode:  8 score -92.00:   average score -95.67
episode:  9 score -94.00:   average score -95.50
episode:  10 score -96.00:   average score -95.55
episode:  11 score -93.00:   average score -95.33
episode:  12 score -89.00:   average score -94.85
episode:  13 score -89.00:   average score -94.43
F2L1 solved: 34
episode ended: 34
episode:  14 score -37.00:   average score -90.60
F2L1 solved: 34
episode ended: 34
episode:  15 score -44.00:   average score -87.69
F2L1 solved: 31
episode ended: 31
episode:  16 score -33.00:   average score -84.47
episode:  17 score -99.00:   average score -85.28
F2L1 sol

In [ ]:
cube = F2L1_cube()
# obs = cube.reset()
cube.scramble("R2 F2 U' B2 D R2 U R2 U L2 B2 R2 L B' D' F2 U' R' B2 U")
# obs, _, _, _ = cube.step(0)
obs = cube.get_f2l1()

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(F2L1_moves[action])
    solution += F2L1_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### F2L1 SOLUTION #####')
print(solution)

L' U L
0 (array([ 1.,  2., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  5.,
        1.,  2., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
L' U L
1 (array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  2., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  2.,  1.,
        5.]), -2.0, False, '')
B U B'
2 (array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  2., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  5.,  2.,  1., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
U
3 (array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  2., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  5.,  2.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -

In [ ]:
cube.get_f2l1()

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  1.,  2., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1.,  2.,  0., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.])

In [ ]:
%load_ext autoreload
%autoreload 2
import importlib
import dqn_agent
importlib.reload(dqn_agent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'dqn_agent' from '/content/dqn_agent.py'>

## Predictions

In [ ]:
from dqn_agent import DQNAgent 

input_oll = 20
input_pll = 12
input_f2l1 = 40
actions_cross = 12
actions_oll = 59
actions_f2l1 = 18
f2l1_solver = dqn_agent.DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_f2l1,
                      n_actions=actions_f2l1, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=model_path)

f2l1_solver.load_model()

 
def predict_f2l1(scramble: str):
    f2l1_cube = F2L1_cube()
    f2l1_cube.scramble(scramble)
    obs = f2l1_cube.get_f2l1()
    i = 0
    solution = ""
    done = False
    while not done:
        action = f2l1_solver.choose_action(obs)
        print(action)
        solution += f2l1_cube.move_list[action] + " "
        next = f2l1_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No f2l1 solution found for this scramble"
    else:
        return solution

In [ ]:
predict_f2l1("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D'")

0
8
7
16
11
F2L1 solved: 5
episode ended: 5


"U L' U L L U' L' F U F' F' U' F "

#Deep Q F2L2


## Training

In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/F2L2_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = F2L2_cube()

    n_games = 150
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_f2l2),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


episode:  0 score -93.00:   average score -93.00
episode:  1 score -93.00:   average score -93.00
episode:  2 score -95.00:   average score -93.67
episode:  3 score -92.00:   average score -93.25
episode:  4 score -93.00:   average score -93.20
episode:  5 score -89.00:   average score -92.50
episode:  6 score -90.00:   average score -92.14
episode:  7 score -90.00:   average score -91.88
episode:  8 score -90.00:   average score -91.67
episode:  9 score -89.00:   average score -91.40
episode:  10 score -97.00:   average score -91.91
episode:  11 score -95.00:   average score -92.17
episode:  12 score -92.00:   average score -92.15
episode:  13 score -91.00:   average score -92.07
episode:  14 score -88.00:   average score -91.80
episode:  15 score -90.00:   average score -91.69
F2L2 solved: 2
episode ended: 2
episode:  16 score 18.00:   average score -85.24
episode:  17 score -94.00:   average score -85.72
episode:  18 score -94.00:   average score -86.16
F2L2 solved: 24
episode ended

In [ ]:
cube = F2L2_cube()
# obs = cube.reset()
cube.scramble("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R'")
# obs, _, _, _ = cube.step(0)
obs = cube.get_f2l2()

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(F2L2_moves[action])
    solution += F2L2_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### F2L2 SOLUTION #####')
print(solution)

U'
0 (array([-1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  3.,  5.,  2., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -1.0, False, '')
B U B'
1 (array([-1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  2.,  5.,  3., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
U'
2 (array([-1., -1., -1., -1., -1., -1.,  2.,  3., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  2.,  5.,  3., -1., -1., -1., -1., -1.,
       -1.]), -1.0, False, '')
F U' F'
3 (array([-1., -1., -1., -1., -1., -1., -1., -1.,  3.,  2., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  2.,  5.,  3., -1., -1.,
       -1.

## Predictions from loaded model

In [ ]:
from dqn_agent import DQNAgent 

input_oll = 20
input_pll = 12
input_f2l2 = 40
actions_cross = 12
actions_oll = 59
actions_f2l2 = len(F2L2_moves)
f2l2_solver = dqn_agent.DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_f2l2,
                      n_actions=actions_f2l2, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=model_path)

f2l2_solver.load_model()

 
def predict_f2l2(scramble: str):
    f2l2_cube = F2L2_cube()
    f2l2_cube.scramble(scramble)
    obs = f2l2_cube.get_f2l2()
    i = 0
    solution = ""
    done = False
    while not done:
        action = f2l2_solver.choose_action(obs)
        solution += f2l2_cube.move_list[action] + " "
        next = f2l2_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No F2L2 solution found for this scramble"
    else:
        return solution

In [ ]:
predict_f2l2("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R'")

F2L2 solved: 6
episode ended: 6


"U' B U B' U' F U' F' L' U L R' U R "

#Deep Q F2L3


## Training

In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/F2L3_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = F2L3_cube()

    n_games = 500
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_f2l3),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


episode:  0 score -88.00:   average score -88.00
episode:  1 score -90.00:   average score -89.00
episode:  2 score -94.00:   average score -90.67
episode:  3 score -94.00:   average score -91.50
episode:  4 score -90.00:   average score -91.20
episode:  5 score -94.00:   average score -91.67
episode:  6 score -90.00:   average score -91.43
episode:  7 score -85.00:   average score -90.62
episode:  8 score -90.00:   average score -90.56
episode:  9 score -87.00:   average score -90.20
episode:  10 score -85.00:   average score -89.73
episode:  11 score -86.00:   average score -89.42
episode:  12 score -89.00:   average score -89.38
episode:  13 score -84.00:   average score -89.00
episode:  14 score -75.00:   average score -88.07
episode:  15 score -82.00:   average score -87.69
episode:  16 score -80.00:   average score -87.24
episode:  17 score -88.00:   average score -87.28
episode:  18 score -81.00:   average score -86.95
episode:  19 score -85.00:   average score -86.85
episode:  

In [ ]:
cube = F2L3_cube()
# obs = cube.reset()
cube.scramble("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R' U' B U B' U' B U B'")
# obs, _, _, _ = cube.step(0)
obs = cube.get_f2l3()

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(F2L3_moves[action])
    solution += F2L3_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### F2L2 SOLUTION #####')
print(solution)

L' U L
0 (array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  4.,  3., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  3.,  5.,
        4.]), -2.0, False, '')
L' U L
1 (array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  4.,  3., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  4.,  3.,  5., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
B' U' B
2 (array([-1., -1., -1., -1., -1., -1.,  3.,  4., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  4.,  3.,  5., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
L' U L
3 (array([-1., -1., -1., -1., -1., -1.,  3.,  4., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  4.,  3.,  5., -1., -1., -1., -1., -1., -1., -1., -1.,
  

## Predictions from loaded model

In [ ]:
from dqn_agent import DQNAgent 

input_oll = 20
input_pll = 12
input_f2l3 = 40
actions_cross = 12
actions_oll = 59
actions_f2l3 = len(F2L3_moves)
f2l3_solver = dqn_agent.DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_f2l3,
                      n_actions=actions_f2l3, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=model_path)

f2l3_solver.load_model()

 
def predict_f2l3(scramble: str):
    f2l3_cube = F2L3_cube()
    f2l3_cube.scramble(scramble)
    obs = f2l3_cube.get_f2l3()
    i = 0
    solution = ""
    done = False
    while not done:
        action = f2l3_solver.choose_action(obs)
        solution += f2l3_cube.move_list[action] + " "
        next = f2l3_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No F2L3 solution found for this scramble"
    else:
        return solution

In [ ]:
predict_f2l3("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U' R U R' U' B U B' U' B U B'")

F2L3 solved: 6
episode ended: 6


"B' U B U F U F' U' F U' F' L U' L' "

#Deep Q F2L4


## Training

In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/F2L4_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = F2L4_cube()

    n_games = 500
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_f2l4),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


episode:  0 score -86.00:   average score -86.00
episode:  1 score -85.00:   average score -85.50
episode:  2 score -84.00:   average score -85.00
episode:  3 score -84.00:   average score -84.75
episode:  4 score -79.00:   average score -83.60
F2L4 solved: 35
episode ended: 35
episode:  5 score -34.00:   average score -75.33
F2L4 solved: 2
episode ended: 2
episode:  6 score 18.00:   average score -62.00
episode:  7 score -79.00:   average score -64.12
episode:  8 score -83.00:   average score -66.22
F2L4 solved: 21
episode ended: 21
episode:  9 score -12.00:   average score -60.80
F2L4 solved: 34
episode ended: 34
episode:  10 score -32.00:   average score -58.18
F2L4 solved: 18
episode ended: 18
episode:  11 score -5.00:   average score -53.75
F2L4 solved: 21
episode ended: 21
episode:  12 score -10.00:   average score -50.38
F2L4 solved: 35
episode ended: 35
episode:  13 score -32.00:   average score -49.07
F2L4 solved: 9
episode ended: 9
episode:  14 score 8.00:   average score -45

In [ ]:
cube = F2L4_cube()
# obs = cube.reset()
cube.scramble("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U'\
                 R U R' U' B U B' U' B U B' F U F' U B' U' B U' L U L'")
# obs, _, _, _ = cube.step(0)
obs = cube.get_f2l4()

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(F2L4_moves[action])
    solution += F2L4_moves[action] + " "
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### F2L2 SOLUTION #####')
print(solution)

U'
0 (array([-1., -1., -1., -1., -1., -1.,  1.,  4., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  4.,  5.,  1., -1., -1.,
       -1.]), -1.0, False, '')
L U L'
1 (array([-1., -1., -1., -1., -1., -1.,  1.,  4., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.,  4.,  1.,  5., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -2.0, False, '')
U
2 (array([-1., -1., -1., -1.,  1.,  4., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.,  4.,  1.,  5., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.]), -1.0, False, '')
L U L'
3 (array([-1., -1.,  1.,  4., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  5.,  1.,  4., -1., -1., -1., -1., -1.,
       -1.])

## Predictions from loaded model

In [ ]:
from dqn_agent import DQNAgent 

input_oll = 20
input_pll = 12
input_f2l4 = 40
actions_cross = 12
actions_oll = 59
actions_f2l4 = len(F2L4_moves)
f2l4_solver = dqn_agent.DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=input_f2l4,
                      n_actions=actions_f2l4, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=model_path)

f2l4_solver.load_model()

 
def predict_f2l4(scramble: str):
    f2l4_cube = F2L4_cube()
    f2l4_cube.scramble(scramble)
    obs = f2l4_cube.get_f2l4()
    i = 0
    solution = ""
    done = False
    while not done:
        action = f2l4_solver.choose_action(obs)
        print(action)
        solution += f2l4_cube.move_list[action] + " "
        next = f2l4_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No F2L4 solution found for this scramble"
    else:
        return solution

In [ ]:
predict_f2l4("U U F L L D D B B D D R R F R R B D D R R U' F D' L U U L' F D' U'\
                 R U R' U' B U B' U' B U B' F U F' U B' U' B U' L U L'")

1
1
3
0
4
2
5
F2L4 solved: 7
episode ended: 7


"U' U' L' U L U F U F' L U L' F U' F' "

#Deep Q OLL


In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/OLL_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = OLL_cube()

    n_games = 2000
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_oll),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=128, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
episode ended: 3
episode:  333 score 14.00:   average score 8.78
OLL solved: 3
episode ended: 3
episode:  334 score 10.00:   average score 8.80
OLL solved: 1
episode ended: 1
episode:  335 score 20.00:   average score 8.92
OLL solved: 5
episode ended: 5
episode:  336 score 12.00:   average score 8.97
OLL solved: 4
episode ended: 4
episode:  337 score 13.00:   average score 9.03
OLL solved: 4
episode ended: 4
episode:  338 score 9.00:   average score 9.03
OLL solved: 6
episode ended: 6
episode:  339 score 3.00:   average score 8.97
OLL solved: 7
episode ended: 7
episode:  340 score -2.00:   average score 8.97
OLL solved: 3
episode ended: 3
episode:  341 score 14.00:   average score 8.93
OLL solved: 4
episode ended: 4
episode:  342 score 13.00:   average score 8.99
OLL solved: 8
episode ended: 8
episode:  343 score 5.00:   average score 8.96
OLL solved: 2
episode ended: 2
episode:  344 score 15.00:   average sc

In [ ]:
cube = OLL_cube()
# obs = cube.reset()
cube.scramble("R2 F R2 B' D2 R2 D2 B R' B' R' D2 F' R' B")
cube.get_oll_state()

array([ 0, -1, -1,  0, -1,  0,  0, -1,  0, -1, -1, -1,  0, -1,  0, -1, -1,
       -1,  0, -1])

In [ ]:
i=0
solution = ""
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(oll_moves[action])
    solution += " " + oll_moves[action]
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1
print('\n ###### OLL SOLUTION #####')
print(solution)

R' F2 R2 U2 R' F R U2 R2 F2 R
0 (array([-1,  0, -1,  0,  0, -1,  0, -1,  0, -1, -1,  0, -1, -1, -1, -1,  0,
        0, -1, -1]), -5.0, False, '')
U
1 (array([-1,  0,  0, -1,  0, -1, -1,  0,  0, -1, -1, -1, -1,  0,  0, -1, -1,
        0, -1, -1]), -1.0, False, '')
U
2 (array([ 0, -1,  0, -1, -1,  0, -1,  0, -1, -1,  0,  0, -1, -1,  0, -1, -1,
        0, -1, -1]), -1.0, False, '')
F U R U' R' F'
3 (array([-1,  0, -1,  0, -1,  0, -1,  0,  0, -1, -1,  0, -1, -1, -1, -1,  0,
        0, -1, -1]), -5.0, False, '')
OLL solved: 5
episode ended: 5
R' F2 R2 U2 R' F R U2 R2 F2 R
4 (array([ 0, -1,  0, -1,  0, -1,  0, -1, -1, -1,  0, -1, -1,  0, -1, -1,  0,
       -1, -1,  0]), 20.0, True, '')

 ###### OLL SOLUTION #####
 R' F2 R2 U2 R' F R U2 R2 F2 R U U F U R U' R' F' R' F2 R2 U2 R' F R U2 R2 F2 R


In [ ]:
agent.save_model()

In [ ]:
from dqn_agent import DQNAgent 

oll_model = '/content/OLL_model_good2k.h5'
input_oll = 20
input_pll = 12
actions_cross = 12
actions_oll = 59
oll_solver = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=input_oll,
                      n_actions=actions_oll, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=oll_model)
oll_solver.load_model()

 
def predict_oll(scramble: str):
    oll_cube = OLL_cube()
    oll_cube.scramble(scramble)
    obs = oll_cube.get_oll_state()
    i = 0
    solution = ""
    done = False
    while not done:
        action = oll_solver.choose_action(obs)
        print(action)
        solution += oll_cube.move_list[action] + " "
        next = oll_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No OLL solution found for this scramble"
    else:
        return solution

In [ ]:
predict_oll("F2 L2 D R2 B2 D' L2 U' R2 U2 R2 L D R2 D' L'")

21
55
10
31
52
5
6
0
42
21
57
57
34
2
41
24
26
15
39
27
25
6
53
16
12
28
21
55
41
1
9
23
15
21
2
43
23
46
3
4
35
55
54
35
44
55
46
26
34
49
26


'No OLL solution found for this scramble'

#Deep Q PLL


In [ ]:
model_path = '/content/drive/MyDrive/Action learning/DQN/PLL_model.h5'

In [ ]:
import numpy as np
from dqn_agent import DQNAgent
# from utils import plot_learning_curve, make_env

if __name__ == '__main__':
    env = PLL_cube()

    n_games = 10000
    best_score = -np.inf
    agent = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=len(env.solved_pll),
                     n_actions=len(env.move_list), mem_size=1_000_000, epsilon_end=0.01,
                     epsilon_dec=0.998, batch_size=64, fname=model_path)

    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        observation = env.reset()
        score = 0

        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            observation = observation_
            agent.learn()

        scores.append(score)
        eps_history.append(agent.epsilon)
        avg_score = np.mean(scores[max(0, i-100): i+1])

        print('episode: ', i, 'score %.2f: ' % score,
              ' average score %.2f' % avg_score)

        if avg_score > best_score:
            agent.save_model()
            best_score = avg_score

        eps_history.append(agent.epsilon)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
episode ended: 4
episode:  8333 score 13.00:   average score 15.19
PLL solved: 2
episode ended: 2
episode:  8334 score 15.00:   average score 15.16
PLL solved: 3
episode ended: 3
episode:  8335 score 14.00:   average score 15.16
PLL solved: 3
episode ended: 3
episode:  8336 score 14.00:   average score 15.17
PLL solved: 2
episode ended: 2
episode:  8337 score 15.00:   average score 15.14
PLL solved: 1
episode ended: 1
episode:  8338 score 20.00:   average score 15.16
PLL solved: 4
episode ended: 4
episode:  8339 score 13.00:   average score 15.16
PLL solved: 3
episode ended: 3
episode:  8340 score 14.00:   average score 15.11
PLL solved: 2
episode ended: 2
episode:  8341 score 15.00:   average score 15.12
PLL solved: 4
episode ended: 4
episode:  8342 score 13.00:   average score 15.11
PLL solved: 3
episode ended: 3
episode:  8343 score 14.00:   average score 15.10
PLL solved: 2
episode ended: 2
episode:  8344

In [ ]:
cube = PLL_cube()
obs = cube.reset()
cube.get_pll_state()

array([4, 3, 2, 3, 4, 4, 1, 2, 1, 2, 1, 3], dtype=int32)

In [ ]:
cube.show2()

       B B B       
       B B B       
       G R G       

O O R  W W W  O R R
O O G  W W W  O R R
O O B  W W W  B R R

       O B R       
       G G G       
       G G G       

       Y Y Y       
       Y Y Y       
       Y Y Y       


In [ ]:
i=0
while not done or i<74:
    action = agent.choose_action(obs)
    next = cube.step(action)
    print(pll_moves[action])
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1

U
0 (array([3, 4, 4, 1, 2, 1, 2, 1, 3, 4, 3, 2], dtype=int32), -1.0, False, '')
R2 U R' U R' U' R U' R2 D U' R' U R D'
1 (array([3, 1, 1, 2, 4, 4, 1, 3, 3, 4, 2, 2], dtype=int32), -5.0, False, '')
U'
2 (array([4, 2, 2, 3, 1, 1, 2, 4, 4, 1, 3, 3], dtype=int32), -1.0, False, '')
U
3 (array([3, 1, 1, 2, 4, 4, 1, 3, 3, 4, 2, 2], dtype=int32), -1.0, False, '')
PLL solved: 5
episode ended: 5
R U R' U R U R' F' R U R' U' R' F R2 U' R' U2 R U' R'
4 (array([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4], dtype=int32), 20.0, True, '')


# Load model in a new agent

In [ ]:
dqn_path = '/content/drive/MyDrive/Action learning/DQN/cross_model15moves-16Jul.h5'

In [ ]:
from tensorflow.keras.models import load_model
model = load_model(dqn_path)

In [ ]:
from dqn_agent import DQNAgent
cross_solver = DQNAgent(gamma=0.99, epsilon=0, alpha=0.0005, input_dims=24,
                     n_actions=12, mem_size=1_000_000, epsilon_end=0.01,
                     batch_size=64, fname=dqn_path, saved_model=model)


In [ ]:
cube = SpeedCube()
# obs = cube.reset()
cube.scramble("R")
obs = cube.get_yellow_edges()
cube.get_yellow_edges()

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,  2,  5,
        1,  5,  4,  5,  3, -1, -1])

In [ ]:
# cube.scramble("U2 F2")
cube.show2()

       B B W       
       B B W       
       B B W       

O O O  W W G  R R R
O O O  W W G  R R R
O O O  W W G  R R R

       G G Y       
       G G Y       
       G G Y       

       Y Y B       
       Y Y B       
       Y Y B       


In [ ]:
i=0
cube._episode_ended = False
done=False
while not done or i<74:
    action = cross_solver.choose_action(obs)
    next = cube.step(action)
    print(move_list[action])
    print(i, next)
    observation_, reward, done, info = next
    if done:
        break
    obs=observation_
    i+=1

R
0 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
        1,  5,  4,  5,  3, -1, -1]), -1.0, False, 'no')
D
1 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
        4,  5,  3, -1, -1,  5,  1]), -1.0, False, 'no')
L
2 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1,  5,  3, -1, -1, -1, -1,  5,
        4, -1, -1, -1, -1,  5,  1]), -1.0, False, 'no')
Lp
3 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
        4,  5,  3, -1, -1,  5,  1]), -1.0, False, 'no')
L
4 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1,  5,  3, -1, -1, -1, -1,  5,
        4, -1, -1, -1, -1,  5,  1]), -1.0, False, 'no')
Lp
5 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
        4,  5,  3, -1, -1,  5,  1]), -1.0, False, 'no')
L
6 (array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1,  5,  3, -1, -1, -1, -1,  5,
        4, -1, -1, -1, -1,  5,  1]), -1.0, False, 'no')
Lp
7 (array([-1, -1,  5,  2, -1, -1, -1, -1, -

In [ ]:
np.argmax(model.predict(obs[np.newaxis, :]))

1

In [ ]:
obs

array([-1, -1,  5,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5,
        4,  5,  3, -1, -1,  5,  1])

In [ ]:
np.argmax(cross_solver.q_eval.predict(obs[np.newaxis, :]))

1

Ideas:
- permutation for + U/U' for F2L
- One F2L at a time
- OLL and PLL, just recognize the case.

Improve cross algo:
- change decay?
- make the Max_sample growing
- other algo

# Other steps

In [44]:
PLL_scrambles = '/content/drive/MyDrive/Action learning/5000_PLL_scrambles.txt'
OLL_scrambles = '/content/drive/MyDrive/Action learning/5000_OLL_scrambles.txt'
F2L_scrambles = '/content/drive/MyDrive/Action learning/5000_F2L_scrambles.txt'
full_scrambles = '/content/drive/MyDrive/Action learning/15000_full_scrambles.txt'
OLL_algs = '/content/drive/MyDrive/Action learning/OLL algs.csv'
PLL_algs =  '/content/drive/MyDrive/Action learning/PLL algs.csv'
daisy = "/content/drive/MyDrive/Action learning/daisy_scrambles.json"

## Process full scrambles to daisy ones

In [45]:
def extract_scramble_list(filepath):
    with open(filepath) as f:
        lines = f.readlines()

    scramble_list = []
    for l in lines:
        scramble = l.strip().split('.')[1][1:]
        scramble_list.append(scramble)
    
    return scramble_list

In [46]:
PLL_scramble_list = extract_scramble_list(PLL_scrambles)
OLL_scramble_list = extract_scramble_list(OLL_scrambles)
F2L_scramble_list = extract_scramble_list(F2L_scrambles) 
full_scramble_list = extract_scramble_list(full_scrambles) 

In [47]:
import pandas as pd
def get_daisy_state(scramble):
    cube = SpeedCube()
    cube.scramble(scramble)
    s = cube.get_daisy()
    return str(s)

daisy_df = pd.DataFrame(full_scramble_list, columns=['scramble'])
daisy_df.head()

,scramble
0,D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F...
1,U2 L2 B2 R2 U' B2 U' L2 R2 D' R F D2 B D L2 D2 B2
2,U2 R' B D' F2 U' L2 R2 U2 F2 R2 D B2 D B R' F'...
3,R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F...
4,F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' ...


In [48]:
daisy_df['status'] = daisy_df['scramble'].apply(get_daisy_state)
daisy_df.iloc[0]['status']
daisy_scrambles_unique = daisy_df.drop_duplicates(subset=['status'])
daisy_scrambles = list(daisy_scrambles_unique['scramble'])
print(len(daisy_scrambles))
print(daisy_scrambles_unique.shape)
daisy_df.shape


6738
(6738, 2)


(15000, 2)

In [49]:
daisy_scrambles_unique['daisy_sol'] = daisy_scrambles_unique['scramble'][200: 600].apply(predict_daisy)

Daisy solved: 6
Daisy solved: 9
Daisy solved: 9
Daisy solved: 8
Daisy solved: 6
Daisy solved: 9
Daisy solved: 9
Daisy solved: 8
Daisy solved: 8
Daisy solved: 8
Daisy solved: 7
Daisy solved: 9
Daisy solved: 6
Daisy solved: 9
Daisy solved: 10
Daisy solved: 8
Daisy solved: 12
Daisy solved: 8
Daisy solved: 8
Daisy solved: 7
Daisy solved: 6
Daisy solved: 6
Daisy solved: 10
Daisy solved: 10
Daisy solved: 6
Daisy solved: 6
Daisy solved: 7
Daisy solved: 6
Daisy solved: 9
Daisy solved: 6
Daisy solved: 7
Daisy solved: 11
Daisy solved: 9
Daisy solved: 8
Daisy solved: 7
Daisy solved: 9
Daisy solved: 8
Daisy solved: 9
Daisy solved: 3
Daisy solved: 7
Daisy solved: 6
Daisy solved: 6
Daisy solved: 9
Daisy solved: 9
Daisy solved: 6
Daisy solved: 9
Daisy solved: 8
Daisy solved: 6
Daisy solved: 7
Daisy solved: 9
Daisy solved: 10
Daisy solved: 9
Daisy solved: 5
Daisy solved: 7
Daisy solved: 6
Daisy solved: 10
Daisy solved: 8
Daisy solved: 8
Daisy solved: 7
Daisy solved: 11
Daisy solved: 6
Daisy solved: 6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
daisy_unique = daisy_scrambles_unique[:200].copy()
daisy_unique.head()

,scramble,status,daisy_sol
0,D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F...,[-1 -1 -1 -1 -1 -1 5 -1 -1 -1 -1 5 -1 -1 -1 ...,B F B U L U L L B
1,U2 L2 B2 R2 U' B2 U' L2 R2 D' R F D2 B D L2 D2 B2,[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 -1 5 -1 ...,F F U B F U U U B U R R R
2,U2 R' B D' F2 U' L2 R2 U2 F2 R2 D B2 D B R' F'...,[-1 -1 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 ...,No daisy solution found for this scramble
3,R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F...,[-1 -1 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 5 -1 ...,L B F R R L R L
4,F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' ...,[ 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 5 -1 -1 ...,L R R R B B


In [31]:
def get_cross(row):
    scramble = row['scramble']
    daisy = row['daisy_sol']
    cube = Daisy_cube()    
    if daisy.startswith('No'):
        return "No solution"
    scramble_s = scramble.strip() + " " + daisy
    cube.scramble(scramble_s)
    s = cube.get_yellow_edges()
    return str(s)

In [32]:
daisy_unique['daisy_state'] = daisy_unique[:200].apply(get_cross, axis=1)

In [35]:
# daisy_scrambles = daisy_unique.drop_duplicates(subset=['daisy_state'])
daisy_scrambles = daisy_unique[~daisy_unique['daisy_sol'].str.startswith("No")]
daisy_scrambles['full_scramble'] = daisy_scrambles.apply(lambda x: x['scramble'].strip()+" "+x['daisy_sol'].strip(), axis=1)
daisy_scrambles.head()
# len(daisy_scrambles)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,scramble,status,daisy_sol,daisy_state,full_scramble
0,D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F...,[-1 -1 -1 -1 -1 -1 5 -1 -1 -1 -1 5 -1 -1 -1 ...,B F B U L U L L B,[ 5 1 5 2 5 4 5 3 -1 -1 -1 -1 -1 -1 -1 ...,D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F...
1,U2 L2 B2 R2 U' B2 U' L2 R2 D' R F D2 B D L2 D2 B2,[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 -1 5 -1 ...,F F U B F U U U B U R R R,[ 5 3 5 2 5 4 5 1 -1 -1 -1 -1 -1 -1 -1 ...,U2 L2 B2 R2 U' B2 U' L2 R2 D' R F D2 B D L2 D2...
3,R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F...,[-1 -1 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 5 -1 ...,L B F R R L R L,[ 5 2 5 1 5 3 5 4 -1 -1 -1 -1 -1 -1 -1 ...,R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F...
4,F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' ...,[ 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 5 -1 -1 ...,L R R R B B,[ 5 3 5 2 5 4 5 1 -1 -1 -1 -1 -1 -1 -1 ...,F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' ...
5,R' D B U2 B2 F2 U' B2 D2 F2 D' R2 F L B' R D L F2,[-1 5 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 5 -1 ...,B L F L R L,[ 5 1 5 3 5 2 5 4 -1 -1 -1 -1 -1 -1 -1 ...,R' D B U2 B2 F2 U' B2 D2 F2 D' R2 F L B' R D L...


In [36]:
daisy_scrambles.shape

(188, 5)

In [37]:
daisy_final = list(daisy_scrambles['full_scramble'])
print(len(daisy_final))
with open('/content/drive/MyDrive/Action learning/daisy_scrambles_unique_new.txt', 'w') as f:
    for d in daisy_final:
        f.write(d)
        f.write('\n')

188


In [ ]:
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

daisy = "/content/drive/MyDrive/Action learning/daisy_scrambles.json"

#### WARNING takes 2h

# for i in tqdm(range(68)):
#     daisy_scrambles_unique['daisy_sol'] = daisy_scrambles_unique['scramble'][i*100: (i+1)*100].apply(predict_daisy)
#     daisy_scrambles_unique.to_json(daisy)

  0%|          | 0/68 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 68/68 [2:10:39<00:00, 115.28s/it]


In [ ]:
import pandas as pd
daisy_df = pd.read_json(daisy)
daisy_df.tail(100)

,scramble,status,daisy_sol
14268,F2 B L2 U2 F' L' B2 R' U2 F2 B2 R2 F2 D' L2 D ...,[-1 -1 -1 5 -1 -1 -1 -1 5 -1 -1 -1 -1 -1 5 ...,None
14287,F' D' R2 F2 D2 B2 U2 B U2 R2 B' L2 B L' B2 U2 ...,[-1 5 -1 -1 -1 5 -1 5 -1 -1 -1 -1 -1 -1 -1 ...,None
14290,L U2 R2 F' U2 R2 B' L2 F L2 R2 B F' R' U L2 B2...,[-1 5 -1 -1 5 -1 -1 -1 -1 -1 -1 5 -1 -1 5 ...,None
14293,B2 U2 L2 B2 R U2 B2 D2 B2 L D2 B2 U' R U2 F' D...,[-1 -1 -1 -1 -1 -1 5 -1 -1 5 -1 -1 -1 -1 5 ...,None
14297,F R' U L' D L F2 D' L2 F2 D2 F U2 R2 F R2 U2 F2 R,[-1 5 5 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...,None
...,...,...,...
14959,U' L' R2 B2 U' L2 F2 D2 F2 R2 U' F2 R2 F R' U'...,[-1 -1 5 -1 -1 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 ...,R Fp L Rp Bp Lp
14972,U R' F2 U2 L F2 L B2 R D2 F2 R2 D' B L U2 L D ...,[-1 -1 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 ...,No daisy solution found for this scramble
14979,R' B D2 F' L2 U2 B L2 U2 F D2 R' U' R' B2 L' U...,[-1 -1 -1 5 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 ...,B R B Rp F Bp Bp L B B
14981,D' L R2 F2 D' F2 R2 U L2 D2 L2 R2 B2 R2 F' L R...,[-1 5 5 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...,No daisy solution found for this scramble


In [ ]:
daisy_scrambles_unique.head(11)

,scramble,status,daisy_sol
0,D' U2 L2 R2 B2 D2 U2 F U2 F D2 L2 F U' R' B' F...,[-1 -1 -1 -1 -1 -1 5 -1 -1 -1 -1 5 -1 -1 -1 ...,F Lp Lp Bp Bp L L L Fp L F Rp Lp
1,U2 L2 B2 R2 U' B2 U' L2 R2 D' R F D2 B D L2 D2 B2,[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 -1 5 -1 ...,No daisy solution found for this scramble
2,U2 R' B D' F2 U' L2 R2 U2 F2 R2 D B2 D B R' F'...,[-1 -1 -1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 ...,No daisy solution found for this scramble
3,R2 U2 L2 D2 F' D2 B D2 F' R2 B D2 R' B' L' U F...,[-1 -1 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 5 -1 ...,B Fp L F Rp Lp
4,F2 D2 R2 D' B2 D2 B2 L2 F2 L2 U2 R' D' F R U' ...,[ 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 5 -1 5 -1 -1 ...,Bp Bp L Rp
5,R' D B U2 B2 F2 U' B2 D2 F2 D' R2 F L B' R D L F2,[-1 5 -1 -1 -1 -1 -1 5 5 -1 -1 -1 -1 5 -1 ...,B B Lp Lp F L Bp R
6,B U2 F2 D2 L2 F2 U2 B' U2 L2 F L' U2 L' B R2 F...,[ 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...,No daisy solution found for this scramble
7,D' F L' F R U' R2 D F R2 F' R2 B' D2 B2 L2 B' ...,[-1 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...,No daisy solution found for this scramble
8,B' L2 B2 U2 F' D2 B' R2 U2 F' U2 R2 U' L2 F' R...,[ 5 -1 5 -1 -1 -1 -1 5 -1 -1 -1 -1 -1 5 -1 ...,No daisy solution found for this scramble
9,B2 D2 L2 F2 U' F2 D R2 U' L2 R D L2 F U2 B L' ...,[ 5 -1 -1 -1 -1 -1 -1 -1 5 -1 -1 5 -1 -1 -1 ...,Bp Lp Bp Bp Lp Bp Bp Rp


In [ ]:
import pandas as pd
def get_f2l1_state(scramble):
    cube = SpeedCube()
    cube.scramble(scramble)
    s = cube.get_f2l1()
    return str(s)

f2l1_df = pd.DataFrame(F2L_scramble_list, columns=['scramble'])
f2l1_df.head()

,scramble
0,U2 F L2 D2 B2 D2 R2 F R2 B D2 R2 U' F D' L U2 ...
1,F2 L U L2 D2 U' B2 U' R2 D2 F L' F L2 U F2 L
2,F2 R' B2 R2 U2 B2 D' B2 L2 B2 F2 U' B2 D2 R' B...
3,R' L U L U2 L2 D' F' D L' U2 B2 L' F2 R2 L' F2...
4,R D' B D R2 F L U L2 U' R2 U2 R2 B2 D' B2 U' F...


In [ ]:
f2l1_df['status'] = f2l1_df['scramble'].apply(get_f2l1_state)
f2l1_df.iloc[0]['status']
f2l1_scrambles = list(f2l1_df.drop_duplicates(subset=['status'])['scramble'])
print(len(f2l1_scrambles))
f2l1_df.shape

384


(5000, 2)

In [ ]:
with open('/content/drive/MyDrive/Action learning/F2L_scrambles_unique.txt', 'w') as f:
    for f2l in f2l1_scrambles:
        f.write(f2l)
        f.write('\n')

In [ ]:


def get_pll_state(scramble):
    cube = SpeedCube()    
    cube.scramble(scramble)
    s = cube.get_pll_state()
    return str(s)

pll_df = pd.DataFrame(PLL_scramble_list, columns=['scramble'])
pll_df.head()

,scramble
0,L2 D R2 B2 D B2 U' B2 U L' R D2 L' R
1,B2 L2 R2 D F2 R2 D L2 D' R2 D' L R' B2 L' R'
2,R2 D B2 D' B2 U B2 U' B2 R2
3,R2 U F2 L2 B2 D' B2 L2 F2 U2 R2
4,L2 B2 U' B2 D L2 D' L2 U L2 U'


In [ ]:
pll_df['status'] = pll_df['scramble'].apply(get_pll_state)
pll_df.head()

,scramble,status
0,L2 D R2 B2 D B2 U' B2 U L' R D2 L' R,[2 1 2 3 4 4 1 2 3 4 3 1]
1,B2 L2 R2 D F2 R2 D L2 D' R2 D' L R' B2 L' R',[1 4 2 3 1 3 4 3 1 2 2 4]
2,R2 D B2 D' B2 U B2 U' B2 R2,[3 1 1 2 3 3 4 4 4 1 2 2]
3,R2 U F2 L2 B2 D' B2 L2 F2 U2 R2,[1 1 1 2 4 2 3 2 3 4 3 4]
4,L2 B2 U' B2 D L2 D' L2 U L2 U',[2 2 4 1 1 2 3 3 3 4 4 1]


In [ ]:
pll_scrambles = list(pll_df.drop_duplicates(subset=['status'])['scramble'])

In [ ]:
with open('/content/drive/MyDrive/Action learning/PLL_scrambles_unique.txt', 'w') as f:
    for pll in pll_scrambles:
        f.write(pll)
        f.write('\n')
        

In [ ]:
import pandas as pd

def get_oll_state(scramble):
    cube = SpeedCube()    
    cube.scramble(scramble)
    s = cube.get_oll_state()
    return str(s)

oll_df = pd.DataFrame(OLL_scramble_list, columns=['scramble'])
oll_df.head()

,scramble
0,F2 R2 B2 D2 L2 R F2 L' R' B R' B2 L2 U2 B' R'
1,U2 B2 L2 B2 L' U2 L2 U' R' U2 L2 U' L' R
2,D' F2 D2 R2 U' R2 D' F2 R2 U' R2 U' F R' F2 R F
3,R' U F2 U2 F2 U F2 U F2 U2 R2 B' R' B R' U' R U
4,L F2 D2 L B2 L B2 L2 D2 F2 R2 F R F' L'


In [ ]:
oll_df['status'] = oll_df['scramble'].apply(get_oll_state)
oll_df.head()

,scramble,status
0,F2 R2 B2 D2 L2 R F2 L' R' B R' B2 L2 U2 B' R',[-1 0 -1 0 -1 0 -1 0 -1 0 -1 0 -1 -1 -1 ...
1,U2 B2 L2 B2 L' U2 L2 U' R' U2 L2 U' L' R,[ 0 -1 0 -1 0 -1 0 -1 -1 0 -1 -1 0 -1 0 ...
2,D' F2 D2 R2 U' R2 D' F2 R2 U' R2 U' F R' F2 R F,[-1 0 -1 0 0 -1 0 -1 -1 -1 0 0 -1 -1 -1 ...
3,R' U F2 U2 F2 U F2 U F2 U2 R2 B' R' B R' U' R U,[-1 0 0 -1 -1 0 0 -1 -1 -1 0 -1 0 -1 -1 ...
4,L F2 D2 L B2 L B2 L2 D2 F2 R2 F R F' L',[-1 0 -1 0 0 -1 0 -1 -1 0 -1 -1 -1 0 0 ...


In [ ]:
oll_scrambles = list(oll_df.drop_duplicates(subset=['status'])['scramble'])
with open('/content/drive/MyDrive/Action learning/OLL_scrambles_unique.txt', 'w') as f:
    for oll in oll_scrambles:
        f.write(oll)
        f.write('\n')

In [ ]:
cube = SpeedCube()
cube.scramble(PLL_scramble_list[0])
cube.show2()

       B B B       
       B B B       
       B R G       

O O O  W W W  O R R
O O B  W W W  O R R
O O G  W W W  B R R

       R G R       
       G G G       
       G G G       

       Y Y Y       
       Y Y Y       
       Y Y Y       


In [ ]:

# cube.show2()
PLL_scramble_list[0]
cube.get_pll_state()

array([2, 1, 2, 3, 4, 4, 1, 2, 3, 4, 3, 1], dtype=int32)

In [ ]:
cube.get_last_layer()

array([0, 1, 0, 4, 0, 2, 0, 3, 2, 3, 0, 4, 1, 0, 3, 4, 0, 1, 2, 0],
      dtype=int32)

In [ ]:
def get_algs(filename):
    with open(filename) as f:
        lines = f.readlines()

    moves = []
    for l in lines[1:]:
        moves.append(l.strip().split(';')[-1])

    return moves

OLL_moves = get_algs(OLL_algs)
PLL_moves = get_algs(PLL_algs)

## Process F2L intermediate scramble (F2L2, F2L3, F2L4)

In [ ]:
import pandas as pd
import json

solutions = pd.read_json('F2L_solutions.json', orient='frame')
solutions.head()

def split_scramble(dict_sol):
    return dict_sol['scramble']

solutions.iloc[0,0]['scramble']
solutions['scramble'] = solutions['solutions'].apply(split_scramble)
solutions['f2l1'] = solutions['solutions'].apply(lambda x: x['solution'][0])
solutions['f2l2'] = solutions['solutions'].apply(lambda x: x['solution'][1])
solutions['f2l3'] = solutions['solutions'].apply(lambda x: x['solution'][2])
solutions['f2l4'] = solutions['solutions'].apply(lambda x: x['solution'][3])
solutions.head()

### concatenate scramble and previous F2L solution

In [ ]:
def F2L2_scramble(x):
    scramble = x['scramble'].copy()
    # print(len(scramble))
    # print(x['f2l1'])
    scramble.extend(x['f2l1'])
    # print(len(scramble))
    return scramble

def F2L3_scramble(x):
    scramble = x['scramble_F2L2'].copy()
    scramble.extend(x['f2l2'])
    return scramble

def F2L4_scramble(x):
    scramble = x['scramble_F2L3'].copy()
    scramble.extend(x['f2l3'])
    return scramble

solutions['scramble_F2L2'] = solutions.apply(F2L2_scramble, axis=1)
solutions['scramble_F2L3'] = solutions.apply(F2L3_scramble, axis=1)
solutions['scramble_F2L4'] = solutions.apply(F2L4_scramble, axis=1)
solutions.head()

### check duplicates and keep unique scrambles

In [ ]:
f2l2_df = pd.DataFrame(solutions['scramble_F2L2'])

def get_f2l2_state(scramble):
    cube = SpeedCube()
    cube.scramble(*scramble)
    s = cube.get_f2l2()
    return str(s)


f2l2_df['state'] = f2l2_df.apply(get_f2l2_state, axis=1)

f2l2_scrambles = list(f2l2_df.drop_duplicates(subset=['state'])['scramble_F2L2'])
print(len(f2l2_scrambles))
f2l2_df.shape

with open('F2L2_scrambles_unique.txt', 'w') as f:
    for f2l in f2l2_scrambles:
        s = " ".join(map(str, f2l))
        f.write(s)
        f.write('\n')

In [ ]:
f2l3_df = pd.DataFrame(f2l_scrambles['scramble_F2L3'])

def get_f2l3_state(scramble):
    cube = SpeedCube()
    cube.scramble(*scramble)
    s = cube.get_f2l3()
    return str(s)


f2l3_df['state'] = f2l3_df.apply(get_f2l3_state, axis=1)

f2l3_scrambles = list(f2l3_df.drop_duplicates(subset=['state'])['scramble_F2L3'])
print(len(f2l3_scrambles))
f2l3_df.shape

with open('F2L3_scrambles_unique.txt', 'w') as f:
    for f2l in f2l3_scrambles:
        s = " ".join(map(str, f2l))
        f.write(s)
        f.write('\n')

In [ ]:
f2l4_df = pd.DataFrame(f2l_scrambles['scramble_F2L4'])

def get_f2l4_state(scramble):
    cube = SpeedCube()
    cube.scramble(*scramble)
    s = cube.get_f2l4()
    return str(s)


f2l4_df['state'] = f2l4_df.apply(get_f2l4_state, axis=1)

f2l4_scrambles = list(f2l4_df.drop_duplicates(subset=['state'])['scramble_F2L4'])
print(len(f2l4_scrambles))
f2l4_df.shape

with open('F2L4_scrambles_unique.txt', 'w') as f:
    for f2l in f2l4_scrambles:
        s = " ".join(map(str, f2l))
        f.write(s)
        f.write('\n')

In [ ]:
# %%time
import requests
import json
from tqdm import tqdm


url = "https://ai-cube-api.herokuapp.com/solve_cube"
# url = "http://127.0.0.1:8000/solve_cube"


success = []
start = 10000
nb_test = 500
good_scramble = []
bad_scramble = []
err = {"cross_errors": 0, "f2l_errors": 0, "oll_errors": 0, "pll_errors": 0}
f2l_errors = {"F2L1": 0, "F2L2": 0, "F2L3": 0, "F2L4": 0}
cross_errors = {"daisy": 0, "cross_from_daisy":0}
for i in tqdm(range(nb_test)): 
    scramble = {"scramble_s": full_scramble_list[i+start]}
    result = requests.post(url=url, json=scramble).json()
    correct = result['solution_found']
    success.append(correct)
    # print(scramble)
    if correct:
        good_scramble.append(full_scramble_list[i+start])
    # else: 
    #     print(result['step_solutions'])
    else:
        bad_scramble.append(full_scramble_list[i+start])
        error = result["step_solutions"]
        if error['Cross'][0] == "No":
            err["cross_errors"] += 1
            if error['Cross'][1] == "Daisy":
                cross_errors['daisy'] +=1
            elif error['Cross'][1] == "Cross":
                cross_errors['cross_from_daisy'] +=1
        elif error["F2L"][0] == "No":
            if error['F2L'][1] == "F2L1":
                f2l_errors['F2L1'] +=1
            elif error['F2L'][1] == "F2L2":
                f2l_errors['F2L2'] +=1
            elif error['F2L'][1] == "F2L3":
                f2l_errors['F2L3'] +=1
            elif error['F2L'][1] == "F2L4":
                f2l_errors['F2L2'] +=1
            err["f2l_errors"] += 1
        elif error["OLL"][0] == "No":
            err["oll_errors"] += 1
        elif error["PLL"][0] == "No":
            err["pll_errors"] += 1

    if i%10 == 0 and i>0: print(f'success {sum(success)}/{i}')

print(f'Success rate {sum(success)/nb_test*100:.2f}%')
print(err)
print(cross_errors)
print(f2l_errors)

  2%|▏         | 11/500 [01:18<57:20,  7.04s/it]

success 9/10


  4%|▍         | 21/500 [02:25<58:42,  7.35s/it]

success 14/20


  6%|▌         | 31/500 [03:14<37:51,  4.84s/it]

...

 28%|██▊       | 141/500 [13:23<32:16,  5.40s/it]

success 108/140


 30%|███       | 151/500 [14:27<31:09,  5.36s/it]

success 113/150


 31%|███       | 153/500 [14:35<27:24,  4.74s/it]

In [54]:
print(cross_errors)
print(f2l_errors)

{'daisy': 7, 'cross_from_daisy': 5}
{'F2L1': 4, 'F2L2': 0, 'F2L3': 0, 'F2L4': 0}


# Inference all

In [ ]:
import numpy as np
import pandas as pd
from typing import List
import json
from dqn_agent import DQNAgent
# from cube_env import OLL_cube, PLL_cube, SpeedCube, MAX_EXPLO


# Loading models
cross_model = "/content/cross_model15moves-16Jul.h5"
oll_model = '/content/0718-16fin_OLL_model.h5'
pll_model = '/content/PLL_model_good10k.h5'

input_cross = 24
input_oll = 20
input_pll = 12
actions_cross = 12
actions_oll = 59
actions_pll = 23

cross_solver = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=input_cross,
                        n_actions=actions_cross, mem_size=1_000_000, epsilon_end=0.01,
                        batch_size=64, fname=cross_model)

cross_solver.load_model()

oll_solver = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=input_oll,
                      n_actions=actions_oll, mem_size=1_000_000, epsilon_end=0.01,
                      epsilon_dec=0.998, batch_size=64, fname=oll_model)
oll_solver.load_model()

pll_solver = DQNAgent(gamma=0.99, epsilon=1.0, alpha=0.0005, input_dims=input_pll,
                      n_actions=actions_pll, mem_size=1_000_000, epsilon_end=0.01,
                      batch_size=64, fname=pll_model)
pll_solver.load_model()


def predict_cross(scramble: str):
    cross_cube = SpeedCube()
    cross_cube.scramble(scramble)
    obs = cross_cube.get_yellow_edges()
    i = 0
    solution = ""
    done = False
    while not done:
        action = cross_solver.choose_action(obs)
        solution += cross_cube.move_list[action] + " "
        next = cross_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No cross solution found for this scramble"
    else:
        return solution


def predict_oll(scramble: str):
    oll_cube = OLL_cube()
    oll_cube.show2()
    oll_cube.scramble(scramble)
    obs = oll_cube.get_oll_state()
    i = 0
    solution = ""
    done = False
    while not done:
        action = oll_solver.choose_action(obs)
        # print(action)
        solution += oll_cube.move_list[action] + " "
        next = oll_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No OLL solution found for this scramble"
    else:
        return solution


def predict_pll(scramble: str):
    pll_cube = PLL_cube()
    pll_cube.scramble(scramble)
    obs = pll_cube.get_pll_state()
    i = 0
    solution = ""
    done = False
    while not done:
        action = pll_solver.choose_action(obs)
        solution += pll_cube.move_list[action] + " "
        next = pll_cube.step(action)
        observation_, _, done, _ = next
        if done:
            break
        obs = observation_
        i += 1
    if i >= MAX_EXPLO:
        return "No PLL solution found for this scramble"
    else:
        return solution


In [ ]:
predict_pll("R2 U F2 L2 B2 D' B2 L2 F2 U2 R2 U2")
# predict_pll("U")

PLL solved: 30
episode ended: 30


"R' U R' U' R D' R' D R' U D' R2 U' R2 D R2 F R U' R' U' R U R' F' R U R' U' R' F R F' R U R' U' D R2 U' R U' R' U R' U R2 D' R' U' F' R U R' U' R' F R2 U' R' U' R U R' U R R' U2 R U2 R' F R U R' U' R' F' R2 R U R' U R' U' R2 U' R' U R' U R R' B' R U' R D R' U R D' R2 B R R' U R U' R' F' U' F R U R' F R' F' R U' R R' B' R U' R D R' U R D' R2 B R R2 U R' U R' U' R U' R2 D U' R' U R D' R' U' R U D' R2 U R' U R U' R U' R2 D L' U' L F L' U' L U L F' L2 U L R' U R' U' R' U' R' U R U R2 U R' U R' U' R D' R' D R' U D' R2 U' R2 D R2 R U R' U' D R2 U' R U' R' U R' U R2 D' L' U' L F L' U' L U L F' L2 U L R' B' R2 D R' U' R D' R' U R' B R R' U2 R U2 R' F R U R' U' R' F' R2 R' B' R U' R D R' U R D' R2 B R R U R' U' D R2 U' R U' R' U R' U R2 D' R2 U' R U' R U R' U R2 D' U R U' R' D R' U' R U D' R2 U R' U R U' R U' R2 D U' R U R' U R U R' F' R U R' U' R' F R2 U' R' U2 R U' R' R U R' U' R' F R2 U' R' U' R U R' F' R U R' F' R U R' U' R' F R2 U' R' R' U' R' D' R U' R' D R U R' D' R U R' D R2 R U R' U R

In [ ]:
predict_oll("R2 F R2 B' D2 R2 D2 B R' B' R' D2 F' R' B")

       B B B       
       B B B       
       B B B       

O O O  W W W  R R R
O O O  W W W  R R R
O O O  W W W  R R R

       G G G       
       G G G       
       G G G       

       Y Y Y       
       Y Y Y       
       Y Y Y       


'No OLL solution found for this scramble'